# Aggregating data with pandas and numpy

## About the Data
In this notebook, we will be working with 2 datasets:
- Facebook's stock price throughout 2018 (obtained using the [`stock_analysis` package](https://github.com/stefmolin/stock-analysis)).
- daily weather data for NYC from the [National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2).

*Note: The NCEI is part of the National Oceanic and Atmospheric Administration (NOAA) and, as you can see from the URL for the API, this resource was created when the NCEI was called the NCDC. Should the URL for this resource change in the future, you can search for "NCEI weather API" to find the updated one.*

## Background on the weather data

Data meanings:
- `AWND`: average wind speed
- `PRCP`: precipitation in millimeters
- `SNOW`: snowfall in millimeters
- `SNWD`: snow depth in millimeters
- `TMAX`: maximum daily temperature in Celsius
- `TMIN`: minimum daily temperature in Celsius

## Setup

In [ ]:
import numpy as np
import pandas as pd

fb = pd.read_csv('data/fb_2018.csv', index_col='date', parse_dates=True).assign(
    trading_volume=lambda x: pd.cut(x.volume, bins=3, labels=['low', 'med', 'high'])
)
fb.head()

In [2]:
weather = pd.read_csv('data/weather_by_station.csv', index_col='date', parse_dates=True)
weather.head()

,datatype,station,value,station_name
date,,,,
2018-01-01,PRCP,GHCND:US1CTFR0039,0.0,"STAMFORD 4.2 S, CT US"
2018-01-01,PRCP,GHCND:US1NJBG0015,0.0,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0015,0.0,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,PRCP,GHCND:US1NJBG0017,0.0,"GLEN ROCK 0.7 SSE, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0017,0.0,"GLEN ROCK 0.7 SSE, NJ US"


In [3]:
weather.shape

(80256, 4)

Before we dive into any calculations, let's make sure `pandas` won't put things in scientific notation. We will modify how floats are formatted for displaying. The format we will apply is `.2f`, which will provide the float with 2 digits after the decimal point:

In [4]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Summarizing DataFrames
We learned about `agg()` in the [`2-dataframe_operations.ipynb`](./2-dataframe_operations.ipynb) notebook when we learned about window calculations; however, we can call this on the dataframe directly to aggregate its contents into a single series:

In [6]:
fb.head()

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.55,181.42,18151903,low
2018-01-03,181.88,184.78,181.33,184.67,16886563,low
2018-01-04,184.90,186.21,184.10,184.33,13880896,low
2018-01-05,185.59,186.90,184.93,186.85,13574535,low
2018-01-08,187.20,188.90,186.33,188.28,17994726,low


In [7]:
#me
fb.agg({'open': ['min', 'max'], 'high':['median'], 'low':'mean'})

,open,high,low
min,123.10,NaN,NaN
max,215.72,NaN,NaN
median,NaN,176.98,NaN
mean,NaN,NaN,169.30


In [8]:
# me
fb.agg({'open': 'mean', 'close':['min', 'mean', 'sum']})

,open,close
mean,171.45,171.51
min,NaN,124.06
sum,NaN,43049.24


In [9]:
fb.agg({
    'open': np.mean, 
    'high': np.max, 
    'low': np.min, 
    'close': np.mean, 
    'volume': np.sum
})

open            171.45
high            218.62
low             123.02
close           171.51
volume   6949682394.00
dtype: float64

We can use this to find the total snowfall and precipitation recorded in Central Park in 2018:

In [10]:
# me
weather.head()

,datatype,station,value,station_name
date,,,,
2018-01-01,PRCP,GHCND:US1CTFR0039,0.00,"STAMFORD 4.2 S, CT US"
2018-01-01,PRCP,GHCND:US1NJBG0015,0.00,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0015,0.00,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,PRCP,GHCND:US1NJBG0017,0.00,"GLEN ROCK 0.7 SSE, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0017,0.00,"GLEN ROCK 0.7 SSE, NJ US"


In [17]:
# me
(
    weather
    .query('station == "GHCND:USW00094728"')
    .pivot(columns='datatype', values='value')
    .loc['2018', ['PRCP', 'SNOW']]
    .agg(sum)
)

datatype
PRCP   1665.30
SNOW   1007.00
dtype: float64

In [25]:
# me
weather.query("datatype in ('SNOW', 'PRCP') and station == 'GHCND:USW00094728'") \
.pivot_table(index = 'date', values = 'value', columns = 'datatype')\
.agg(sum)

datatype
PRCP   1665.30
SNOW   1007.00
dtype: float64

In [20]:
#me
weather.query("datatype in ('SNOW', 'PRCP')") \
.query('station == "GHCND:USW00094728"') \
.pivot(columns = 'datatype', values = 'value').head()

datatype,PRCP,SNOW
date,,
2018-01-01,0.00,0.00
2018-01-02,0.00,0.00
2018-01-03,0.00,0.00
2018-01-04,19.30,249.00
2018-01-05,0.00,0.00


In [21]:
#me
weather.query("datatype in ('SNOW', 'PRCP')") \
.query('station == "GHCND:USW00094728"') \
.pivot(columns = 'datatype', values = 'value') \
.agg(sum)

datatype
PRCP   1665.30
SNOW   1007.00
dtype: float64

In [22]:
# me
weather.query("datatype in ('SNOW', 'PRCP')") \
.query('station == "GHCND:USW00094728"') \
.pivot_table(values = 'value', columns = 'datatype', aggfunc=sum)


datatype,PRCP,SNOW
value,1665.30,1007.00


In [26]:
weather.query('station == "GHCND:USW00094728"')\
    .pivot(columns='datatype', values='value')[['SNOW', 'PRCP']]\
    .sum()

datatype
SNOW   1007.00
PRCP   1665.30
dtype: float64

This is equivalent to passing `'sum'` to `agg()`:

In [27]:
weather.query('station == "GHCND:USW00094728"')\
    .pivot(columns='datatype', values='value')[['SNOW', 'PRCP']]\
    .agg('sum')

datatype
SNOW   1007.00
PRCP   1665.30
dtype: float64

Note that we aren't limited to providing a single aggregation per column. We can pass a list, and we will get a dataframe back instead of a series. Null values are placed where we don't have a calculation result to display:

In [28]:
fb.agg({
    'open': 'mean',
    'high': ['min', 'max'],
    'low': ['min', 'max'],
    'close': 'mean'
})

,open,high,low,close
mean,171.45,NaN,NaN,171.51
min,NaN,129.74,123.02,NaN
max,NaN,218.62,214.27,NaN


## Using `groupby()`
Often we won't want to aggregate on the entire dataframe, but on groups within it. For this purpose, we can run `groupby()` before the aggregation. If we group by the `trading_volume` column, we will get a row for each of the values it takes on:

In [29]:
fb.head()

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.55,181.42,18151903,low
2018-01-03,181.88,184.78,181.33,184.67,16886563,low
2018-01-04,184.90,186.21,184.10,184.33,13880896,low
2018-01-05,185.59,186.90,184.93,186.85,13574535,low
2018-01-08,187.20,188.90,186.33,188.28,17994726,low


In [30]:
# me
(
    fb
    .groupby('trading_volume')
    .agg({'open': 'min', 'high':[np.mean, np.sum], 'low':'median'})
)

open   high             low
                  min   mean      sum median
trading_volume                              
low            123.10 173.46 41630.65 173.00
med            156.31 179.42  1435.36 168.31
high           160.82 170.48   511.43 161.95

In [31]:
# me
fb.groupby('trading_volume').mean()

,open,high,low,close,volume
trading_volume,,,,,
low,171.36,173.46,169.31,171.43,24547207.71
med,175.82,179.42,172.11,175.14,79072559.12
high,167.73,170.48,161.57,168.16,141924023.33


In [32]:
# me
fb.groupby('trading_volume').agg({'open':[np.min, np.max], 'close':np.sum, 'volume':[np.min, np.mean, np.sum] })

open           close     volume                         
                 amin   amax      sum       amin         mean         sum
trading_volume                                                           
low            123.10 215.11 41143.62    9588587  24547207.71  5891329851
med            156.31 215.72  1401.15   64592585  79072559.12   632580473
high           160.82 174.89   504.47  126116634 141924023.33   425772070

In [33]:
fb.groupby('trading_volume').mean()

,open,high,low,close,volume
trading_volume,,,,,
low,171.36,173.46,169.31,171.43,24547207.71
med,175.82,179.42,172.11,175.14,79072559.12
high,167.73,170.48,161.57,168.16,141924023.33


After we call `groupby()`, we can still select columns for aggregation:

In [34]:
fb.groupby('trading_volume')['close'].agg(['min', 'max', 'mean'])

,min,max,mean
trading_volume,,,
low,124.06,214.67,171.43
med,152.22,217.50,175.14
high,160.06,176.26,168.16


We can still provide a dictionary specifying the aggregations to perform, but passing a list for a column will result in a hierarchical index for the columns:

In [42]:
fb_agg = fb.groupby('trading_volume').agg({
    'open': 'mean',
    'high': ['min', 'max'],
    'low': ['min', 'max'],
    'close': 'mean'
})
fb_agg

open   high           low         close
                 mean    min    max    min    max   mean
trading_volume                                          
low            171.36 129.74 216.20 123.02 212.60 171.43
med            175.82 162.85 218.62 150.75 214.27 175.14
high           167.73 161.10 180.13 149.02 173.75 168.16

The hierarchical index in the columns looks like this:

In [43]:
fb_agg.columns

MultiIndex([( 'open', 'mean'),
            ( 'high',  'min'),
            ( 'high',  'max'),
            (  'low',  'min'),
            (  'low',  'max'),
            ('close', 'mean')],
           )

Using a list comprehension, we can join the levels (in a tuple) with an `_` at each iteration: 

In [44]:
new_columns = ['_'.join(x) for x in fb_agg.columns]
new_columns

['open_mean', 'high_min', 'high_max', 'low_min', 'low_max', 'close_mean']

In [45]:
fb_new = fb_agg.copy()
fb_new.columns = new_columns
fb_new

,open_mean,high_min,high_max,low_min,low_max,close_mean
trading_volume,,,,,,
low,171.36,129.74,216.20,123.02,212.60,171.43
med,175.82,162.85,218.62,150.75,214.27,175.14
high,167.73,161.10,180.13,149.02,173.75,168.16


In [46]:
fb_agg.columns = ['_'.join(col_agg) for col_agg in fb_agg.columns]
fb_agg.head()

,open_mean,high_min,high_max,low_min,low_max,close_mean
trading_volume,,,,,,
low,171.36,129.74,216.20,123.02,212.60,171.43
med,175.82,162.85,218.62,150.75,214.27,175.14
high,167.73,161.10,180.13,149.02,173.75,168.16


We can group on values in the index if we tell `groupby()`, which `level` to use:

In [47]:
weather.head()

,datatype,station,value,station_name
date,,,,
2018-01-01,PRCP,GHCND:US1CTFR0039,0.00,"STAMFORD 4.2 S, CT US"
2018-01-01,PRCP,GHCND:US1NJBG0015,0.00,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0015,0.00,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,PRCP,GHCND:US1NJBG0017,0.00,"GLEN ROCK 0.7 SSE, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0017,0.00,"GLEN ROCK 0.7 SSE, NJ US"


In [ ]:
# me
weather.loc['2018-10'].query('datatype == "PRCP"')['value'] \
.groupby(level=0).mean().head(13)

In [50]:
# what does the above do? I think it averages PRCP per day, across stations. 

# start w/the first part of the code above:
weather.loc['2018-10'].query('datatype == "PRCP"').head()

# we see multiple stations reporting PRCP

,datatype,station,value,station_name
date,,,,
2018-10-01,PRCP,GHCND:US1CTFR0039,0.00,"STAMFORD 4.2 S, CT US"
2018-10-01,PRCP,GHCND:US1NJBG0003,0.00,"TENAFLY 1.3 W, NJ US"
2018-10-01,PRCP,GHCND:US1NJBG0017,0.00,"GLEN ROCK 0.7 SSE, NJ US"
2018-10-01,PRCP,GHCND:US1NJBG0018,0.00,"PALISADES PARK 0.2 WNW, NJ US"
2018-10-01,PRCP,GHCND:US1NJBG0023,0.00,"OAKLAND 0.9 SSE, NJ US"


In [52]:
# select the 'value' column and then group by level=0, which means the index, which in this case...is the date.
weather.loc['2018-10'].query('datatype == "PRCP"')['value']\
.groupby(level=0).mean().head(13)

date
2018-10-01    0.01
2018-10-02    2.23
2018-10-03   19.69
2018-10-04    0.32
2018-10-05    0.97
2018-10-06    0.06
2018-10-07    0.67
2018-10-08    0.16
2018-10-09    1.04
2018-10-10    0.15
2018-10-11    3.00
2018-10-12   34.77
2018-10-13    1.90
Name: value, dtype: float64

In [53]:
weather.loc['2018-10'].query('datatype == "PRCP"')\
    .groupby(level=0).mean(numeric_only=True).head().squeeze()

date
2018-10-01    0.01
2018-10-02    2.23
2018-10-03   19.69
2018-10-04    0.32
2018-10-05    0.97
Name: value, dtype: float64

In [ ]:
# I THINK squeeze turns a single series dataframe into a series

In [54]:
type(weather.loc['2018-10'].query('datatype == "PRCP"')\
    .groupby(level=0).mean(numeric_only=True).head())

pandas.core.frame.DataFrame

In [55]:
type(weather.loc['2018-10'].query('datatype == "PRCP"')\
    .groupby(level=0).mean(numeric_only=True).head().squeeze())

pandas.core.series.Series

In [ ]:
# YUP!

We can also create a `Grouper` object, which can also roll up the datetimes in the index. Here, we find the quarterly total precipitation per station:

In [56]:
#me
weather.head()

,datatype,station,value,station_name
date,,,,
2018-01-01,PRCP,GHCND:US1CTFR0039,0.00,"STAMFORD 4.2 S, CT US"
2018-01-01,PRCP,GHCND:US1NJBG0015,0.00,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0015,0.00,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,PRCP,GHCND:US1NJBG0017,0.00,"GLEN ROCK 0.7 SSE, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0017,0.00,"GLEN ROCK 0.7 SSE, NJ US"


In [64]:
# me
(
    weather
    .query('datatype == "PRCP"')
    .groupby([pd.Grouper(freq='Q'), 'station'])['value']
    .sum()
)

date        station          
2018-03-31  GHCND:US1CTFR0039   338.00
            GHCND:US1NJBG0003   319.20
            GHCND:US1NJBG0010    67.60
            GHCND:US1NJBG0015   327.70
            GHCND:US1NJBG0017   289.80
                                 ...  
2018-12-31  GHCND:USW00054787   417.20
            GHCND:USW00094728   450.30
            GHCND:USW00094741   450.20
            GHCND:USW00094745   434.70
            GHCND:USW00094789   454.50
Name: value, Length: 397, dtype: float64

In [65]:
# me...adding unstack to the above might make it easier to read
(
    weather
    .query('datatype == "PRCP"')
    .groupby([pd.Grouper(freq='Q'), 'station'])['value']
    .sum()
    .unstack()
)

station,GHCND:US1CTFR0039,GHCND:US1NJBG0003,GHCND:US1NJBG0010,GHCND:US1NJBG0015,GHCND:US1NJBG0017,GHCND:US1NJBG0018,GHCND:US1NJBG0023,GHCND:US1NJBG0030,GHCND:US1NJBG0037,GHCND:US1NJBG0039,...,GHCND:USC00301309,GHCND:USC00308577,GHCND:USW00014732,GHCND:USW00014734,GHCND:USW00054743,GHCND:USW00054787,GHCND:USW00094728,GHCND:USW00094741,GHCND:USW00094745,GHCND:USW00094789
date,,,,,,,,,,,,,,,,,,,,,
2018-03-31,338.00,319.20,67.60,327.70,289.80,325.20,379.90,352.00,NaN,299.60,...,327.20,297.10,308.80,312.30,215.10,230.80,334.80,333.00,238.00,326.50
2018-06-30,272.10,269.40,319.30,321.90,321.90,341.70,354.30,292.50,NaN,257.80,...,190.90,274.00,301.70,292.60,233.50,240.60,315.50,322.60,299.60,286.00
2018-09-30,424.70,586.60,664.40,496.10,432.10,624.10,514.10,573.50,378.70,563.70,...,551.00,446.70,452.80,452.80,590.40,402.90,564.70,505.50,540.10,391.80
2018-12-31,390.00,415.50,181.90,393.30,404.00,309.60,477.30,490.80,284.00,330.40,...,495.70,529.40,432.20,453.70,357.10,417.20,450.30,450.20,434.70,454.50


In [67]:
# me...reversing the order in groupby helps even more
(
    weather
    .query('datatype == "PRCP"')
    .groupby(['station', pd.Grouper(freq='Q')])['value']
    .sum()
    .unstack()
)

date,2018-03-31,2018-06-30,2018-09-30,2018-12-31
station,,,,
GHCND:US1CTFR0039,338.00,272.10,424.70,390.00
GHCND:US1NJBG0003,319.20,269.40,586.60,415.50
GHCND:US1NJBG0010,67.60,319.30,664.40,181.90
GHCND:US1NJBG0015,327.70,321.90,496.10,393.30
GHCND:US1NJBG0017,289.80,321.90,432.10,404.00
...,...,...,...,...
GHCND:USW00054787,230.80,240.60,402.90,417.20
GHCND:USW00094728,334.80,315.50,564.70,450.30
GHCND:USW00094741,333.00,322.60,505.50,450.20


In [66]:
# me
weather.query('datatype == "PRCP"') \
.groupby( ['station', pd.Grouper(freq = 'Q')]).sum(numeric_only=True) \
.unstack().sample(5, random_state=1)

value                                 
date              2018-03-31 2018-06-30 2018-09-30 2018-12-31
station                                                      
GHCND:USC00308577     297.10     274.00     446.70     529.40
GHCND:US1NYSF0092     385.70     254.40     471.70     470.20
GHCND:US1NYWC0019     285.30     214.00     376.40     403.60
GHCND:US1NYSF0089     314.90     259.60     342.60     383.80
GHCND:USW00014732     308.80     301.70     452.80     432.20

In [71]:
weather.query('datatype == "PRCP"').groupby(
    ['station', pd.Grouper(freq='Q')]
).sum(numeric_only=True).unstack().sample(5, random_state=1)

value                                 
date              2018-03-31 2018-06-30 2018-09-30 2018-12-31
station                                                      
GHCND:USC00308577     297.10     274.00     446.70     529.40
GHCND:US1NYSF0092     385.70     254.40     471.70     470.20
GHCND:US1NYWC0019     285.30     214.00     376.40     403.60
GHCND:US1NYSF0089     314.90     259.60     342.60     383.80
GHCND:USW00014732     308.80     301.70     452.80     432.20

Note that we can use `filter()` to exclude some groups from aggregation. Here, we only keep groups with names ending in "NY US" in the group's `name` attribute, which is the station name in this case:

In [72]:
# me

weather.groupby('station_name') \
.filter( lambda x: x.name.endswith("NY US"))

# THE X IS THE ITEM GROUPED ON. IN THIS CASE, 'station_name'

,datatype,station,value,station_name
date,,,,
2018-01-01,PRCP,GHCND:US1NYKN0025,0.00,"BROOKLYN 3.1 NW, NY US"
2018-01-01,SNOW,GHCND:US1NYKN0025,0.00,"BROOKLYN 3.1 NW, NY US"
2018-01-01,WESF,GHCND:US1NYKN0025,0.00,"BROOKLYN 3.1 NW, NY US"
2018-01-01,PRCP,GHCND:US1NYNS0007,0.00,"FLORAL PARK 0.4 W, NY US"
2018-01-01,SNOW,GHCND:US1NYNS0007,0.00,"FLORAL PARK 0.4 W, NY US"
...,...,...,...,...
2018-12-31,WDF5,GHCND:USW00094789,130.00,"JFK INTERNATIONAL AIRPORT, NY US"
2018-12-31,WSF2,GHCND:USW00094789,9.80,"JFK INTERNATIONAL AIRPORT, NY US"
2018-12-31,WSF5,GHCND:USW00094789,12.50,"JFK INTERNATIONAL AIRPORT, NY US"


In [73]:
weather.groupby('station_name').filter( # station names with "NY US" in them
    lambda x: x.name.endswith('NY US')
).query('datatype == "SNOW"').groupby('station_name').sum(numeric_only=True).squeeze() # aggregate and make a series (squeeze)

station_name
ALBERTSON 0.2 SSE, NY US           1087.00
AMITYVILLE 0.1 WSW, NY US           434.00
AMITYVILLE 0.6 NNE, NY US          1072.00
ARMONK 0.3 SE, NY US               1504.00
BROOKLYN 3.1 NW, NY US              305.00
CENTERPORT 0.9 SW, NY US            799.00
CENTERPORT, NY US                  1333.00
ELMSFORD 0.8 SSW, NY US             863.00
FLORAL PARK 0.4 W, NY US           1015.00
HICKSVILLE 1.3 ENE, NY US           716.00
JACKSON HEIGHTS 0.3 WSW, NY US      107.00
JFK INTERNATIONAL AIRPORT, NY US    855.00
LA GUARDIA AIRPORT, NY US           892.00
LOCUST VALLEY 0.3 E, NY US            0.00
LYNBROOK 0.3 NW, NY US              325.00
MASSAPEQUA 0.9 SSW, NY US            41.00
MIDDLE VILLAGE 0.5 SW, NY US       1249.00
NEW HYDE PARK 1.6 NE, NY US           0.00
NEW YORK 8.8 N, NY US                 0.00
NORTH WANTAGH 0.4 WSW, NY US        471.00
NY CITY CENTRAL PARK, NY US        1007.00
PLAINEDGE 0.4 WSW, NY US            610.00
PLAINVIEW 0.4 ENE, NY US           1360.0

In [74]:
 # me ---> I'm curious what happens without the second groupby
weather.groupby('station_name').filter( # station names with "NY US" in them
    lambda x: x.name.endswith('NY US')
).query('datatype == "SNOW"').head()

,datatype,station,value,station_name
date,,,,
2018-01-01,SNOW,GHCND:US1NYKN0025,0.00,"BROOKLYN 3.1 NW, NY US"
2018-01-01,SNOW,GHCND:US1NYNS0007,0.00,"FLORAL PARK 0.4 W, NY US"
2018-01-01,SNOW,GHCND:US1NYNS0024,0.00,"NORTH WANTAGH 0.4 WSW, NY US"
2018-01-01,SNOW,GHCND:US1NYNS0034,0.00,"WANTAGH 0.3 ESE, NY US"
2018-01-01,SNOW,GHCND:US1NYNS0035,0.00,"VALLEY STREAM 0.6 SE, NY US"


Let's see which months have the most precipitation. First, we need to group by day and average the precipitation across the stations. Then we can group by month and sum the resulting precipitation. We use `nlargest()` to give the 5 months with the most precipitation:

In [76]:
weather.head()

,datatype,station,value,station_name
date,,,,
2018-01-01,PRCP,GHCND:US1CTFR0039,0.00,"STAMFORD 4.2 S, CT US"
2018-01-01,PRCP,GHCND:US1NJBG0015,0.00,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0015,0.00,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,PRCP,GHCND:US1NJBG0017,0.00,"GLEN ROCK 0.7 SSE, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0017,0.00,"GLEN ROCK 0.7 SSE, NJ US"


In [93]:
(
    weather
    .query('datatype == "PRCP"')
    #.groupby(pd.Grouper(freq='1D'))  --> These two versions of groupby are interchangeable
    .groupby(level=0)
    .mean(numeric_only=True)
    .groupby(pd.Grouper(freq='M'))
    .sum()
    .nlargest(columns='value', n=12)

)

,value
date,
2018-11-30,210.59
2018-09-30,193.09
2018-08-31,192.45
2018-07-31,160.98
2018-02-28,158.11
2018-12-31,157.75
2018-04-30,140.57
2018-03-31,137.46
2018-05-31,113.38


In [94]:
# me
weather.query('datatype == "PRCP"') \
.rename({'value':'Monthly_PRCP'}, axis=1) \
.groupby(level=0).mean(numeric_only=True) \
.groupby(pd.Grouper(freq='M')).sum().sort_values(by = 'Monthly_PRCP', ascending = False)

,Monthly_PRCP
date,
2018-11-30,210.59
2018-09-30,193.09
2018-08-31,192.45
2018-07-31,160.98
2018-02-28,158.11
2018-12-31,157.75
2018-04-30,140.57
2018-03-31,137.46
2018-05-31,113.38


In [95]:
weather.query('datatype == "PRCP"')\
    .groupby(level=0).mean()\
    .groupby(pd.Grouper(freq='M')).sum().value.nlargest()

<ipython-input-95-9d83b13b874a>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  weather.query('datatype == "PRCP"')\


date
2018-11-30   210.59
2018-09-30   193.09
2018-08-31   192.45
2018-07-31   160.98
2018-02-28   158.11
Name: value, dtype: float64

Perhaps the previous result was surprising. The saying goes "April showers bring May flowers"; yet April wasn't in the top 5 (neither was May for that matter). Snow will count towards precipitation, but that doesn't explain why summer months are higher than April. Let's look for days that accounted for a large percentage of the precipitation in a given month. 

In order to do so, we need to calculate the average daily precipitation across stations and then find the total per month. This will be the denominator. However, in order to divide the daily values by the total for their month, we will need a series of equal dimensions. This means we will need to use `transform()`:

In [96]:
weather.head()

,datatype,station,value,station_name
date,,,,
2018-01-01,PRCP,GHCND:US1CTFR0039,0.00,"STAMFORD 4.2 S, CT US"
2018-01-01,PRCP,GHCND:US1NJBG0015,0.00,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0015,0.00,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,PRCP,GHCND:US1NJBG0017,0.00,"GLEN ROCK 0.7 SSE, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0017,0.00,"GLEN ROCK 0.7 SSE, NJ US"


In [120]:
# me
# This solution requires me to turn a series back into a dataframe . Seems inelegant. 

PRCP_by_day = (
    weather
    .query('datatype == "PRCP"')
    .groupby(level=0)['value']
    .mean()
    .rename('PRCP_Daily')
    .to_frame()
    .assign(
        PRCP_Monthly = lambda x: x.PRCP_Daily.groupby(pd.Grouper(freq='M')).transform(sum),
        PRCP_PTT = lambda x: x.PRCP_Daily.div(x.PRCP_Monthly).mul(100)
   
    )
    
)
PRCP_by_day.head(10)

,PRCP_Daily,PRCP_Monthly,PRCP_PTT
date,,,
2018-01-01,0.00,69.31,0.00
2018-01-02,0.00,69.31,0.00
2018-01-03,0.00,69.31,0.00
2018-01-04,4.92,69.31,7.09
2018-01-05,12.30,69.31,17.75
2018-01-06,0.03,69.31,0.05
2018-01-07,0.00,69.31,0.00
2018-01-08,0.10,69.31,0.14
2018-01-09,0.30,69.31,0.44


In [117]:
PRCP_by_day.loc['2018-01'].sum()

PRCP_Daily       69.31
PRCP_Monthly   2148.72
PRCP_PTT        100.00
dtype: float64

In [132]:
# me
# This solution eliminates the need for to_frame by calling mean with numeric_only, so the groupby isn't turned into a series
    weather
    .query('datatype == "PRCP"')
    .groupby(level=0)
    .mean(numeric_only=True)
    .rename({'value':'PRCP_Daily'}, axis=1)
    .assign(
        PRCP_Monthly = lambda x: x.PRCP_Daily.groupby(pd.Grouper(freq='M')).transform(sum),
        PRCP_PTT = lambda x: x.PRCP_Daily.div(x.PRCP_Monthly).mul(100)
   
    )
)
     
    
PRCP_by_day.head(10)

,PRCP_Daily,PRCP_Monthly,PRCP_PTT
date,,,
2018-01-01,0.00,69.31,0.00
2018-01-02,0.00,69.31,0.00
2018-01-03,0.00,69.31,0.00
2018-01-04,4.92,69.31,7.09
2018-01-05,12.30,69.31,17.75
2018-01-06,0.03,69.31,0.05
2018-01-07,0.00,69.31,0.00
2018-01-08,0.10,69.31,0.14
2018-01-09,0.30,69.31,0.44


In [133]:
# me
weather.query('datatype == "PRCP"') \
.rename({'value':'PRCP'}, axis=1) \
.groupby(level = 0).mean(numeric_only = True) \
.assign(
    PRCP_M = lambda x: x.groupby(pd.Grouper(freq = 'M')).transform(np.sum),
    PRCP_PCT = lambda x: x.PRCP.div(x.PRCP_M)
).nlargest(10, 'PRCP_PCT')

,PRCP,PRCP_M,PRCP_PCT
date,,,
2018-10-12,34.77,105.63,0.33
2018-01-13,21.66,69.31,0.31
2018-03-02,38.77,137.46,0.28
2018-04-16,39.34,140.57,0.28
2018-04-17,37.30,140.57,0.27
2018-09-26,45.84,193.09,0.24
2018-03-08,32.38,137.46,0.24
2018-12-21,35.41,157.75,0.22
2018-11-25,40.85,210.59,0.19


In [134]:
weather.query('datatype == "PRCP"')\
    .rename(dict(value='prcp'), axis=1)\
    .groupby(level=0).mean()\
    .groupby(pd.Grouper(freq='M'))\
    .transform(np.sum)['2018-01-28':'2018-02-03']

<ipython-input-134-da13659fcc45>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  weather.query('datatype == "PRCP"')\


,prcp
date,
2018-01-28,69.31
2018-01-29,69.31
2018-01-30,69.31
2018-01-31,69.31
2018-02-01,158.11
2018-02-02,158.11
2018-02-03,158.11


Notice how we have the same value repeated for each day in the month it belongs to. This will allow us to calculate the percentage of the monthly precipitation that occurred each day and then pull out the largest values:

In [135]:
weather\
    .query('datatype == "PRCP"')\
    .rename(dict(value='prcp'), axis=1)\
    .groupby(level=0).mean()\
    .assign(
        total_prcp_in_month=lambda x: \
            x.groupby(pd.Grouper(freq='M')).transform(np.sum),
        pct_monthly_prcp=lambda x: \
            x.prcp.div(x.total_prcp_in_month)
    )\
    .nlargest(5, 'pct_monthly_prcp')

<ipython-input-135-529b8b8f0d49>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  weather\


,prcp,total_prcp_in_month,pct_monthly_prcp
date,,,
2018-10-12,34.77,105.63,0.33
2018-01-13,21.66,69.31,0.31
2018-03-02,38.77,137.46,0.28
2018-04-16,39.34,140.57,0.28
2018-04-17,37.30,140.57,0.27


`transform()` can be used on dataframes as well. We can use it to easily standardize the data:

In [139]:
fb.head()

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.55,181.42,18151903,low
2018-01-03,181.88,184.78,181.33,184.67,16886563,low
2018-01-04,184.90,186.21,184.10,184.33,13880896,low
2018-01-05,185.59,186.90,184.93,186.85,13574535,low
2018-01-08,187.20,188.90,186.33,188.28,17994726,low


In [140]:
(
    fb[['open', 'high', 'low', 'close']]
    .transform(lambda x: x.sub(x.mean()).div(x.std()))
)

,open,high,low,close
date,,,,
2018-01-02,0.32,0.41,0.41,0.50
2018-01-03,0.53,0.57,0.60,0.66
2018-01-04,0.68,0.65,0.74,0.64
2018-01-05,0.72,0.68,0.78,0.77
2018-01-08,0.80,0.79,0.85,0.84
...,...,...,...,...
2018-12-24,-2.46,-2.26,-2.31,-2.38
2018-12-26,-2.31,-2.03,-2.16,-1.87
2018-12-27,-1.98,-1.99,-1.97,-1.85


In [136]:
# me
fb[['open', 'high', 'low', 'close']] \
.transform(lambda x: x.sub(x.mean()).div(x.std()) )

,open,high,low,close
date,,,,
2018-01-02,0.32,0.41,0.41,0.50
2018-01-03,0.53,0.57,0.60,0.66
2018-01-04,0.68,0.65,0.74,0.64
2018-01-05,0.72,0.68,0.78,0.77
2018-01-08,0.80,0.79,0.85,0.84
...,...,...,...,...
2018-12-24,-2.46,-2.26,-2.31,-2.38
2018-12-26,-2.31,-2.03,-2.16,-1.87
2018-12-27,-1.98,-1.99,-1.97,-1.85


In [ ]:
fb[['open', 'high', 'low', 'close']]\
    .transform(lambda x: (x - x.mean()).div(x.std()))\
    .head()

## Pivot tables and crosstabs
We saw pivots in [`ch_03/4-reshaping_data.ipynb`](../ch_03/4-reshaping_data.ipynb); however, we weren't able to provide any aggregations. With `pivot_table()`, we get the mean by default. In its simplest form, we provide a column to place along the columns:

In [141]:
fb.head()

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.55,181.42,18151903,low
2018-01-03,181.88,184.78,181.33,184.67,16886563,low
2018-01-04,184.90,186.21,184.10,184.33,13880896,low
2018-01-05,185.59,186.90,184.93,186.85,13574535,low
2018-01-08,187.20,188.90,186.33,188.28,17994726,low


In [142]:
# me
fb.pivot_table(columns='trading_volume')

trading_volume,low,med,high
close,171.43,175.14,168.16
high,173.46,179.42,170.48
low,169.31,172.11,161.57
open,171.36,175.82,167.73
volume,24547207.71,79072559.12,141924023.33


In [144]:
# me. Above, same as
fb.pivot_table(columns='trading_volume', aggfunc='mean')


trading_volume,low,med,high
close,171.43,175.14,168.16
high,173.46,179.42,170.48
low,169.31,172.11,161.57
open,171.36,175.82,167.73
volume,24547207.71,79072559.12,141924023.33


In [137]:
fb.pivot_table(columns='trading_volume')

trading_volume,low,med,high
close,171.43,175.14,168.16
high,173.46,179.42,170.48
low,169.31,172.11,161.57
open,171.36,175.82,167.73
volume,24547207.71,79072559.12,141924023.33


By placing the trading volume in the index, we get the transpose:

In [145]:
#me
fb.pivot_table(index='trading_volume')

,close,high,low,open,volume
trading_volume,,,,,
low,171.43,173.46,169.31,171.36,24547207.71
med,175.14,179.42,172.11,175.82,79072559.12
high,168.16,170.48,161.57,167.73,141924023.33


In [146]:
#me
fb.groupby('trading_volume').mean()

,open,high,low,close,volume
trading_volume,,,,,
low,171.36,173.46,169.31,171.43,24547207.71
med,175.82,179.42,172.11,175.14,79072559.12
high,167.73,170.48,161.57,168.16,141924023.33


In [147]:
fb.pivot_table(index='trading_volume')

,close,high,low,open,volume
trading_volume,,,,,
low,171.43,173.46,169.31,171.36,24547207.71
med,175.14,179.42,172.11,175.82,79072559.12
high,168.16,170.48,161.57,167.73,141924023.33


With `pivot()`, we also weren't able to handle multi-level indices or indices with repeated values. For this reason we haven't been able to put the weather data in the wide format. The `pivot_table()` method solves this issue:

In [148]:
weather.head()

,datatype,station,value,station_name
date,,,,
2018-01-01,PRCP,GHCND:US1CTFR0039,0.00,"STAMFORD 4.2 S, CT US"
2018-01-01,PRCP,GHCND:US1NJBG0015,0.00,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0015,0.00,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,PRCP,GHCND:US1NJBG0017,0.00,"GLEN ROCK 0.7 SSE, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0017,0.00,"GLEN ROCK 0.7 SSE, NJ US"


In [149]:
# this looks...had to work with because of the date

(
    weather
    .reset_index()
    .pivot_table(
        index = ['date', 'station', 'station_name'], 
        columns='datatype', 
        values='value')
                       
)

datatype                                                          AWND  DAPR  \
date       station           station_name                                      
2018-01-01 GHCND:US1CTFR0039 STAMFORD 4.2 S, CT US                 NaN   NaN   
           GHCND:US1NJBG0015 NORTH ARLINGTON 0.7 WNW, NJ US        NaN   NaN   
           GHCND:US1NJBG0017 GLEN ROCK 0.7 SSE, NJ US              NaN   NaN   
           GHCND:US1NJBG0018 PALISADES PARK 0.2 WNW, NJ US         NaN   NaN   
           GHCND:US1NJBG0023 OAKLAND 0.9 SSE, NJ US                NaN   NaN   
...                                                                ...   ...   
2018-12-31 GHCND:USW00054787 FARMINGDALE REPUBLIC AIRPORT, NY US  5.00   NaN   
           GHCND:USW00094728 NY CITY CENTRAL PARK, NY US           NaN   NaN   
           GHCND:USW00094741 TETERBORO AIRPORT, NJ US             1.70   NaN   
           GHCND:USW00094745 WESTCHESTER CO AIRPORT, NY US        2.70   NaN   
           GHCND:USW00094789 JFK INTERNATIONAL AIRPORT, NY US     4.10   NaN   

datatype                                                          MDPR  \
date       station           station_name                                
2018-01-01 GHCND:US1CTFR0039 STAMFORD 4.2 S, CT US                 NaN   
           GHCND:US1NJBG0015 NORTH ARLINGTON 0.7 WNW, NJ US        NaN   
           GHCND:US1NJBG0017 GLEN ROCK 0.7 SSE, NJ US              NaN   
           GHCND:US1NJBG0018 PALISADES PARK 0.2 WNW, NJ US         NaN   
           GHCND:US1NJBG0023 OAKLAND 0.9 SSE, NJ US                NaN   
...                                                                ...   
2018-12-31 GHCND:USW00054787 FARMINGDALE REPUBLIC AIRPORT, NY US   NaN   
           GHCND:USW00094728 NY CITY CENTRAL PARK, NY US           NaN   
           GHCND:USW00094741 TETERBORO AIRPORT, NJ US              NaN   
           GHCND:USW00094745 WESTCHESTER CO AIRPORT, NY US         NaN   
           GHCND:USW00094789 JFK INTERNATIONAL AIRPORT, NY US      NaN   

datatype                                                            PGTM  \
date       station           station_name                                  
2018-01-01 GHCND:US1CTFR0039 STAMFORD 4.2 S, CT US                   NaN   
           GHCND:US1NJBG0015 NORTH ARLINGTON 0.7 WNW, NJ US          NaN   
           GHCND:US1NJBG0017 GLEN ROCK 0.7 SSE, NJ US                NaN   
           GHCND:US1NJBG0018 PALISADES PARK 0.2 WNW, NJ US           NaN   
           GHCND:US1NJBG0023 OAKLAND 0.9 SSE, NJ US                  NaN   
...                                                                  ...   
2018-12-31 GHCND:USW00054787 FARMINGDALE REPUBLIC AIRPORT, NY US 2052.00   
           GHCND:USW00094728 NY CITY CENTRAL PARK, NY US             NaN   
           GHCND:USW00094741 TETERBORO AIRPORT, NJ US            1954.00   
           GHCND:USW00094745 WESTCHESTER CO AIRPORT, NY US       2212.00   
           GHCND:USW00094789 JFK INTERNATIONAL AIRPORT, NY US        NaN   

datatype                                                          PRCP  SNOW  \
date       station           station_name                                      
2018-01-01 GHCND:US1CTFR0039 STAMFORD 4.2 S, CT US                0.00   NaN   
           GHCND:US1NJBG0015 NORTH ARLINGTON 0.7 WNW, NJ US       0.00  0.00   
           GHCND:US1NJBG0017 GLEN ROCK 0.7 SSE, NJ US             0.00  0.00   
           GHCND:US1NJBG0018 PALISADES PARK 0.2 WNW, NJ US        0.00  0.00   
           GHCND:US1NJBG0023 OAKLAND 0.9 SSE, NJ US               0.00  0.00   
...                                                                ...   ...   
2018-12-31 GHCND:USW00054787 FARMINGDALE REPUBLIC AIRPORT, NY US 28.70   NaN   
           GHCND:USW00094728 NY CITY CENTRAL PARK, NY US         25.90  0.00   
           GHCND:USW00094741 TETERBORO AIRPORT, NJ US            29.20   NaN   
           GHCND:USW00094745 WESTCHESTER CO AIRPORT, NY US       24.40   NaN   
           GHCND:USW00094789 JFK INTERNATIONAL 

In [150]:
# add reset_index again to get the multi_index into columns
(
    weather
    .reset_index()
    .pivot_table(
        index = ['date', 'station', 'station_name'], 
        columns='datatype', 
        values='value')
    .reset_index()
                       
)

datatype,date,station,station_name,AWND,DAPR,MDPR,PGTM,PRCP,SNOW,SNWD,...,WSF5,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT11
0,2018-01-01,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",NaN,NaN,NaN,NaN,0.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,GHCND:US1NJBG0015,"NORTH ARLINGTON 0.7 WNW, NJ US",NaN,NaN,NaN,NaN,0.00,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01,GHCND:US1NJBG0017,"GLEN ROCK 0.7 SSE, NJ US",NaN,NaN,NaN,NaN,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01,GHCND:US1NJBG0018,"PALISADES PARK 0.2 WNW, NJ US",NaN,NaN,NaN,NaN,0.00,0.00,3.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-01,GHCND:US1NJBG0023,"OAKLAND 0.9 SSE, NJ US",NaN,NaN,NaN,NaN,0.00,0.00,25.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28740,2018-12-31,GHCND:USW00054787,"FARMINGDALE REPUBLIC AIRPORT, NY US",5.00,NaN,NaN,2052.00,28.70,NaN,NaN,...,15.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28741,2018-12-31,GHCND:USW00094728,"NY CITY CENTRAL PARK, NY US",NaN,NaN,NaN,NaN,25.90,0.00,0.00,...,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28742,2018-12-31,GHCND:USW00094741,"TETERBORO AIRPORT, NJ US",1.70,NaN,NaN,1954.00,29.20,NaN,NaN,...,8.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28743,2018-12-31,GHCND:USW00094745,"WESTCHESTER CO AIRPORT, NY US",2.70,NaN,NaN,2212.00,24.40,NaN,NaN,...,11.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
weather.reset_index().pivot_table(
    index=['date', 'station', 'station_name'], 
    columns='datatype', 
    values='value',
    aggfunc='median'
).reset_index().tail()

datatype,date,station,station_name,AWND,DAPR,MDPR,PGTM,PRCP,SNOW,SNWD,...,WSF5,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT11
28740,2018-12-31,GHCND:USW00054787,"FARMINGDALE REPUBLIC AIRPORT, NY US",5.00,NaN,NaN,2052.00,28.70,NaN,NaN,...,15.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28741,2018-12-31,GHCND:USW00094728,"NY CITY CENTRAL PARK, NY US",NaN,NaN,NaN,NaN,25.90,0.00,0.00,...,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28742,2018-12-31,GHCND:USW00094741,"TETERBORO AIRPORT, NJ US",1.70,NaN,NaN,1954.00,29.20,NaN,NaN,...,8.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28743,2018-12-31,GHCND:USW00094745,"WESTCHESTER CO AIRPORT, NY US",2.70,NaN,NaN,2212.00,24.40,NaN,NaN,...,11.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28744,2018-12-31,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",4.10,NaN,NaN,NaN,31.20,0.00,0.00,...,12.50,1.00,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
# me
weather.reset_index().head(3)

,date,datatype,station,value,station_name
0,2018-01-01,PRCP,GHCND:US1CTFR0039,0.00,"STAMFORD 4.2 S, CT US"
1,2018-01-01,PRCP,GHCND:US1NJBG0015,0.00,"NORTH ARLINGTON 0.7 WNW, NJ US"
2,2018-01-01,SNOW,GHCND:US1NJBG0015,0.00,"NORTH ARLINGTON 0.7 WNW, NJ US"


In [153]:
# me
weather.reset_index().pivot_table(
    index=['date', 'station', 'station_name'], 
    columns='datatype', 
    values='value',
    aggfunc='median'
).head(3)

datatype                                                     AWND  DAPR  MDPR  \
date       station           station_name                                       
2018-01-01 GHCND:US1CTFR0039 STAMFORD 4.2 S, CT US            NaN   NaN   NaN   
           GHCND:US1NJBG0015 NORTH ARLINGTON 0.7 WNW, NJ US   NaN   NaN   NaN   
           GHCND:US1NJBG0017 GLEN ROCK 0.7 SSE, NJ US         NaN   NaN   NaN   

datatype                                                     PGTM  PRCP  SNOW  \
date       station           station_name                                       
2018-01-01 GHCND:US1CTFR0039 STAMFORD 4.2 S, CT US            NaN  0.00   NaN   
           GHCND:US1NJBG0015 NORTH ARLINGTON 0.7 WNW, NJ US   NaN  0.00  0.00   
           GHCND:US1NJBG0017 GLEN ROCK 0.7 SSE, NJ US         NaN  0.00  0.00   

datatype                                                     SNWD  TAVG  TMAX  \
date       station           station_name                                       
2018-01-01 GHCND:US1CTFR0039 STAMFORD 4.2 S, CT US            NaN   NaN   NaN   
           GHCND:US1NJBG0015 NORTH ARLINGTON 0.7 WNW, NJ US   NaN   NaN   NaN   
           GHCND:US1NJBG0017 GLEN ROCK 0.7 SSE, NJ US        0.00   NaN   NaN   

datatype                                                     TMIN  ...  WSF5  \
date       station           station_name                          ...         
2018-01-01 GHCND:US1CTFR0039 STAMFORD 4.2 S, CT US            NaN  ...   NaN   
           GHCND:US1NJBG0015 NORTH ARLINGTON 0.7 WNW, NJ US   NaN  ...   NaN   
           GHCND:US1NJBG0017 GLEN ROCK 0.7 SSE, NJ US         NaN  ...   NaN   

datatype                                                     WT01  WT02  WT03  \
date       station           station_name                                       
2018-01-01 GHCND:US1CTFR0039 STAMFORD 4.2 S, CT US            NaN   NaN   NaN   
           GHCND:US1NJBG0015 NORTH ARLINGTON 0.7 WNW, NJ US   NaN   NaN   NaN   
           GHCND:US1NJBG0017 GLEN ROCK 0.7 SSE, NJ US         NaN   NaN   NaN   

datatype                                                     WT04  WT05  WT06  \
date       station           station_name                                       
2018-01-01 GHCND:US1CTFR0039 STAMFORD 4.2 S, CT US            NaN   NaN   NaN   
           GHCND:US1NJBG0015 NORTH ARLINGTON 0.7 WNW, NJ US   NaN   NaN   NaN   
           GHCND:US1NJBG0017 GLEN ROCK 0.7 SSE, NJ US         NaN   NaN   NaN   

datatype                                                     WT08  WT09  WT11  
date       station           station_name                                      
2018-01-01 GHCND:US1CTFR0039 STAMFORD 4.2 S, CT US            NaN   NaN   NaN  
           GHCND:US1NJBG0015 NORTH ARLINGTON 0.7 WNW, NJ US   NaN   NaN   NaN  
           GHCND:US1NJBG0017 GLEN ROCK 0.7 SSE, NJ US         NaN   NaN   NaN  

[3 rows x 27 columns]

In [154]:
# me
weather.reset_index().pivot_table(
    index=['date', 'station', 'station_name'], 
    columns='datatype', 
    values='value',
    aggfunc='median'
).reset_index().tail()

datatype,date,station,station_name,AWND,DAPR,MDPR,PGTM,PRCP,SNOW,SNWD,...,WSF5,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT11
28740,2018-12-31,GHCND:USW00054787,"FARMINGDALE REPUBLIC AIRPORT, NY US",5.00,NaN,NaN,2052.00,28.70,NaN,NaN,...,15.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28741,2018-12-31,GHCND:USW00094728,"NY CITY CENTRAL PARK, NY US",NaN,NaN,NaN,NaN,25.90,0.00,0.00,...,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28742,2018-12-31,GHCND:USW00094741,"TETERBORO AIRPORT, NJ US",1.70,NaN,NaN,1954.00,29.20,NaN,NaN,...,8.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28743,2018-12-31,GHCND:USW00094745,"WESTCHESTER CO AIRPORT, NY US",2.70,NaN,NaN,2212.00,24.40,NaN,NaN,...,11.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28744,2018-12-31,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",4.10,NaN,NaN,NaN,31.20,0.00,0.00,...,12.50,1.00,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can use the `pd.crosstab()` function to create a frequency table. For example, if we want to see how many low-, medium-, and high-volume trading days Facebook stock had each month, we can use crosstab:

In [155]:
pd.crosstab(
    index=fb.trading_volume,
    columns=fb.index.month,
    colnames=['month'] # name the columns index
)

month,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,20,19,15,20,22,21,18,23,19,23,21,19
med,1,0,4,1,0,0,2,0,0,0,0,0
high,0,0,2,0,0,0,1,0,0,0,0,0


We can normalize with the row or column totals with the `normalize` parameter. This shows percentage of the total:

In [156]:
pd.crosstab(
    index=fb.trading_volume,
    columns=fb.index.month,
    colnames=['month'],
    normalize='columns'
)

month,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,0.95,1.00,0.71,0.95,1.00,1.00,0.86,1.00,1.00,1.00,1.00,1.00
med,0.05,0.00,0.19,0.05,0.00,0.00,0.10,0.00,0.00,0.00,0.00,0.00
high,0.00,0.00,0.10,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00


If we want to perform a calculation other than counting the frequency, we can pass the column to run the calculation on to `values` and the function to use to `aggfunc`:

In [157]:
pd.crosstab(
    index=fb.trading_volume,
    columns=fb.index.month,
    colnames=['month'],
    values=fb.close,
    aggfunc=np.mean
)

month,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,185.24,180.27,177.07,163.29,182.93,195.27,201.92,177.49,164.38,154.19,141.64,137.16
med,179.37,NaN,164.76,174.16,NaN,NaN,194.28,NaN,NaN,NaN,NaN,NaN
high,NaN,NaN,164.11,NaN,NaN,NaN,176.26,NaN,NaN,NaN,NaN,NaN


We can also get row and column subtotals with the `margins` parameter. Let's count the number of times each station recorded snow per month and include the subtotals:

In [158]:
snow_data = weather.query('datatype == "SNOW"')
pd.crosstab(
    index=snow_data.station_name,
    columns=snow_data.index.month,
    colnames=['month'],
    values=snow_data.value,
    aggfunc=lambda x: (x > 0).sum(),
    margins=True, # show row and column subtotals
    margins_name='total observations of snow' # name the subtotals
)

month,1,2,3,4,5,6,7,8,9,10,11,12,total observations of snow
station_name,,,,,,,,,,,,,
"ALBERTSON 0.2 SSE, NY US",3.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,9
"AMITYVILLE 0.1 WSW, NY US",1.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3
"AMITYVILLE 0.6 NNE, NY US",3.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8
"ARMONK 0.3 SE, NY US",6.00,4.00,6.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,3.00,23
"BLOOMINGDALE 0.7 SSE, NJ US",2.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"WESTFIELD 0.6 NE, NJ US",3.00,0.00,4.00,1.00,0.00,NaN,0.00,0.00,0.00,NaN,1.00,NaN,9
"WOODBRIDGE TWP 1.1 ESE, NJ US",4.00,1.00,3.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,11
"WOODBRIDGE TWP 1.1 NNE, NJ US",2.00,1.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,7


<hr>
<div>
    <a href="./2-dataframe_operations.ipynb">
        <button>&#8592; Previous Notebook</button>
    </a>
    <a href="./4-time_series.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<hr>